<a href="https://colab.research.google.com/github/congnhatdt03-hue/Nhan_dien_bien_so_xe/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_web_video_ngrok__%C4%90%C3%A3_ho%C3%A0n_thi%E1%BB%87n_b%C3%A1o_c%C3%A1o_c%C3%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q streamlit opencv-python pillow ultralytics easyocr google-generativeai cloudflared



In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!mkdir -p /content/project/models
!cp "/content/drive/MyDrive/đồ án 2/best.pt" /content/project/models/
!cp "/content/drive/MyDrive/đồ án 2/sort.py" /content/project

In [5]:
!pip install filterpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=f11d58c6158b064d7832a88578a0592844ada48259657f84a85cf7e1233e99e9
  Stored in directory: /root/.cache/pip/wheels/77/bf/4c/b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy


In [33]:
%%writefile /content/project/app.py
import streamlit as st
import cv2
import numpy as np
import re
import os
import tempfile
from PIL import Image
from ultralytics import YOLO
import easyocr
from collections import Counter
import sys
sys.path.append('/content/sort')  # Thêm nếu chưa có
from sort import Sort
import subprocess
import torch  # Thêm để check GPU
from difflib import SequenceMatcher  # Thêm cho merge (nếu dùng similarity, nhưng ở đây dùng IoU)
import pandas as pd  # Thêm cho export CSV

st.set_page_config(page_title="Nhận diện biển số xe", layout="wide")

# =================== HÀM HỖ TRỢ ===================
def iou(b1, b2):
    x1, y1, x2, y2 = b1
    X1, Y1, X2, Y2 = b2
    inter_x1, inter_y1 = max(x1, X1), max(y1, Y1)
    inter_x2, inter_y2 = min(x2, X2), min(y2, Y2)
    iw, ih = max(0, inter_x2 - inter_x1), max(0, inter_y2 - inter_y1)
    inter = iw * ih
    a1 = (x2 - x1) * (y2 - y1)
    a2 = (X2 - X1) * (Y2 - Y1)
    return inter / (a1 + a2 - inter + 1e-6)

def sharpness_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# ====== HÀM KIỂM TRA BIỂN SỐ HỢP LỆ ======
VALID_PLATE_REGEX = re.compile(r"^(\d{2}|T\d{1,2})[- ]?([A-Z]{1,2})[- ]?(\d{2,6})$")
VALID_PROVINCES = {
    "11", "12", "14", "15", "16", "17", "18", "19", "20",
    "21", "22", "23", "24", "25", "26", "27", "28", "29",
    "30", "31", "32", "33", "34", "35", "36", "37", "38",
    "40", "41", "43", "47", "49", "50", "51", "52", "53",
    "54", "55", "56", "57", "58", "59", "60", "61", "62",
    "63", "64", "65", "66", "67", "68", "69", "70", "72",
    "73", "74", "75", "76", "77", "78", "79", "81", "82",
    "83", "84", "85", "86", "88", "89", "90", "92", "93",
    "94", "95", "97", "98", "99"
}

def is_valid_plate(plate: str) -> bool:
    return bool(VALID_PLATE_REGEX.match(plate))

def is_valid_province(plate: str) -> bool:
    match = re.match(r"^(\d{2})", plate)
    return bool(match and (match.group(1) in VALID_PROVINCES or plate.startswith("T")))

def is_valid_vietnam_plate(plate: str) -> bool:
    if not is_valid_plate(plate):
        return False
    return is_valid_province(plate)

# ====== CONFIG ======
YOLO_PATH = "/content/drive/MyDrive/đồ án 2/best.pt"
FRAME_STEP = 200  # ms default
CONFIDENCE_THRESHOLD = 0.25  # default
USE_EASYOCR_FALLBACK = True
IOU_TH = 0.3  # Giảm từ 0.6 để tránh tách track
MAX_AGE = 50  # default

# Sidebar với bố cục gọn gàng
st.sidebar.title("Cài đặt ứng dụng")

# Phần nhập API Key
st.sidebar.header("🔑 Nhập API Key")
GEMINI_API_KEY = st.sidebar.text_input("Nhập Gemini API Key:", type="password")

# Phần chọn phiên bản Gemini
st.sidebar.header("🌐 Chọn phiên bản Gemini")
GEMINI_MODEL_OPTIONS = {
    "gemini-1.5-flash": "~50 requests/ngày",
    "gemini-1.5-pro": "~15 requests/ngày",
    "gemini-2.0-flash": "~1000 requests/ngày",
    "gemini-2.5-flash": "~1500 requests/ngày",
    "gemini-2.5-pro": "~50-100 requests/ngày"
}
selected_gemini_model = st.sidebar.selectbox(
    "Chọn mô hình Gemini:",
    options=list(GEMINI_MODEL_OPTIONS.keys()),
    index=list(GEMINI_MODEL_OPTIONS.keys()).index("gemini-2.5-flash")  # Default là gemini-2.5-flash
)
st.sidebar.write(f"Quota: {GEMINI_MODEL_OPTIONS[selected_gemini_model]}")

# Phần cài đặt tham số động
st.sidebar.header("⚙️ Cài đặt tham số")
CONFIDENCE_THRESHOLD = st.sidebar.slider("Ngưỡng tin cậy YOLO", 0.1, 0.9, CONFIDENCE_THRESHOLD)
FRAME_STEP = st.sidebar.slider("Khoảng cách detect (ms)", 100, 500, FRAME_STEP)
MAX_AGE = st.sidebar.slider("Max age SORT", 10, 100, MAX_AGE)
IOU_TH = st.sidebar.slider("IoU threshold SORT", 0.1, 0.8, IOU_TH)

# Cấu hình Gemini dựa trên lựa chọn
if GEMINI_API_KEY:
    import google.generativeai as genai
    genai.configure(api_key=GEMINI_API_KEY)
    gemini_model = genai.GenerativeModel(selected_gemini_model)
else:
    gemini_model = None

# ====== LOAD YOLO MODEL ======
@st.cache_resource
def load_yolo(path):
    model = YOLO(path)
    return model
model = load_yolo(YOLO_PATH)

# ====== REGEX CHUẨN HÓA ======
FALLBACK_REGEX = re.compile(r"(\d{2})\s*[- ]?\s*([A-Z]{1,2})\s*[- ]?\s*([0-9]{2,6})")
def normalize_plate(text: str) -> str:
    text = text.upper()
    text = re.sub(r"[^A-Z0-9]", "", text)
    match = re.match(r"^(\d{2})([A-Z]{1,2})(\d{2,6})$", text)
    if match:
        p1, p2, p3 = match.groups()
        return f"{p1}-{p2} {p3}"
    match2 = FALLBACK_REGEX.search(text)
    if match2:
        p1, p2, p3 = match2.groups()
        return f"{p1}-{p2} {p3}"
    return text

# ====== OCR ======
def ocr_with_gemini(image: Image.Image) -> str:
    try:
        prompt = "Đọc biển số xe trong ảnh (chỉ trả lời biển số, không thêm ký tự nào khác):"
        res = gemini_model.generate_content([prompt, image])
        txt = res.text.strip()
        return normalize_plate(txt)
    except Exception as e:
        st.warning(f"Lỗi Gemini: {e}")
        return ""

@st.cache_resource
def load_easyocr():
    return easyocr.Reader(['en'])

def ocr_with_easyocr(image: np.ndarray) -> str:
    reader = load_easyocr()
    results = reader.readtext(image)
    txt = " ".join([r[1] for r in results])
    return normalize_plate(txt)

# ====== TIỀN XỬ LÝ ROI ======
def enhance_roi(roi: np.ndarray) -> np.ndarray:
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(gray)
    sharp = cv2.addWeighted(clahe, 1.5, cv2.GaussianBlur(clahe, (0,0), 1), -0.5, 0)
    return cv2.cvtColor(sharp, cv2.COLOR_GRAY2BGR)

# ====== HÀM MERGE TRACKS ======
def merge_tracks(tracks):
    track_ids = sorted(tracks.keys())  # Sort để merge theo thứ tự tid
    i = 0
    while i < len(track_ids):
        tid_i = track_ids[i]
        if tid_i not in tracks:
            i += 1
            continue

        j = i + 1
        while j < len(track_ids):
            tid_j = track_ids[j]
            if tid_j not in tracks:
                j += 1
                continue

            # Check IoU giữa bbox cuối i và bbox đầu j
            if tracks[tid_i]["boxes_history"] and tracks[tid_j]["boxes_history"]:
                last_box_i = tracks[tid_i]["boxes_history"][-1]
                first_box_j = tracks[tid_j]["boxes_history"][0]
                if iou(last_box_i, first_box_j) > 0.5:  # Threshold merge
                    # Merge j vào i
                    tracks[tid_i]["boxes_history"].extend(tracks[tid_j]["boxes_history"])
                    if tracks[tid_j]["best_score"] > tracks[tid_i]["best_score"]:
                        tracks[tid_i]["best_roi"] = tracks[tid_j]["best_roi"]
                        tracks[tid_i]["best_score"] = tracks[tid_j]["best_score"]
                    del tracks[tid_j]
                    track_ids.remove(tid_j)  # Cập nhật list
                else:
                    j += 1
        i += 1
    return tracks

# ====== HIỂN THỊ GIAO DIỆN ======
st.title("Nhận diện biển số xe từ ảnh hoặc video")
st.markdown("<div style='text-align:center; font-size:18px; color:gray;'>Đồ án II – GVHD: ThS. Nguyễn Thị Huế</div>", unsafe_allow_html=True)

input_type = st.radio("Chọn loại dữ liệu:", ["Ảnh", "Video"])

if input_type == "Ảnh":
    uploaded_file = st.file_uploader("📤 Tải lên ảnh", type=["jpg", "jpeg", "png"])
else:
    uploaded_file = st.file_uploader("📤 Tải lên video", type=["mp4", "mov", "avi"])

if uploaded_file and st.button("🚀 Bắt đầu nhận diện"):
    plate_counter = Counter()

    # ================= XỬ LÝ ẢNH =================
    if input_type == "Ảnh":
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        result = model.predict(image, conf=CONFIDENCE_THRESHOLD, half=torch.cuda.is_available())[0]  # Thêm half nếu GPU
        annotated = image.copy()

        if len(result.boxes) == 0:
            st.warning("❌ Không có biển số được phát hiện.")
        else:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                roi = annotated[y1:y2, x1:x2]
                roi = enhance_roi(roi)

                if gemini_model:
                    pil_roi = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
                    txt_gem = ocr_with_gemini(pil_roi)
                    txt_easy = ocr_with_easyocr(roi)
                    # Chọn cái khớp regex tốt hơn
                    if re.match(r"^\d{2}-[A-Z]{1,2} \d{2,6}$", txt_gem):
                        text = txt_gem
                    else:
                        text = txt_easy
                else:
                    text = ocr_with_easyocr(roi) if USE_EASYOCR_FALLBACK else ""

                if is_valid_vietnam_plate(text):
                    plate_counter[text] += 1
                else:
                    st.info(f"🚫 Bỏ qua chuỗi không hợp lệ: {text}")
                cv2.rectangle(annotated, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.putText(annotated, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

            st.image(annotated, caption="Ảnh đã nhận diện", channels="BGR", use_column_width=True)

    # ================= XỬ LÝ VIDEO =================
    else:
        # Lưu file tạm
        temp_video = tempfile.NamedTemporaryFile(delete=False)
        temp_video.write(uploaded_file.read())
        temp_video_path = temp_video.name

        cap = cv2.VideoCapture(temp_video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps * FRAME_STEP / 1000) if FRAME_STEP > 0 else 1
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out_path = "output_annotated.mp4"
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        pbar = st.progress(0.0)
        ocr_status = st.empty()

        # ================= PASS 1: TRACK + LẤY ROI =================
        sort_tracker = Sort(max_age=MAX_AGE, min_hits=1, iou_threshold=IOU_TH)
        tracks = {}  # tid -> {"best_roi": np.array, "best_score": float, "text": str, "boxes_history": list}
        box_found = False
        idx = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # YOLO detect theo frame_interval
            if idx % frame_interval == 0:
                result = model.predict(frame, conf=CONFIDENCE_THRESHOLD, verbose=False, half=torch.cuda.is_available())[0]
                dets = np.array([
                    [*box.xyxy[0].cpu().numpy(), box.conf[0].cpu().numpy()]
                    for box in result.boxes
                ]) if len(result.boxes) > 0 else np.empty((0, 5))

                if len(dets) > 0:
                    box_found = True
            else:
                dets = np.empty((0, 5))

            # SORT update
            trackers = sort_tracker.update(dets)

            for track in trackers:
                x1, y1, x2, y2, tid = map(int, track)
                if tid not in tracks:
                    tracks[tid] = {
                        "best_roi": None,
                        "best_score": -1.0,
                        "text": "",
                        "boxes_history": []
                    }

                tracks[tid]["boxes_history"].append([x1, y1, x2, y2])
                if len(tracks[tid]["boxes_history"]) > 15:  # Tăng từ 7 lên 15
                    tracks[tid]["boxes_history"].pop(0)

                # Luôn crop ROI từ SORT box
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0:
                    continue

                # Nếu YOLO bỏ sót, confirm ROI bằng YOLO phụ với margin
                matched = any(iou([x1, y1, x2, y2], det[:4]) > IOU_TH for det in dets)
                if not matched:
                    margin = 20  # pixel mở rộng
                    exp_y1, exp_y2 = max(0, y1 - margin), min(height, y2 + margin)
                    exp_x1, exp_x2 = max(0, x1 - margin), min(width, x2 + margin)
                    exp_roi = frame[exp_y1:exp_y2, exp_x1:exp_x2]
                    sub_result = model.predict(exp_roi, conf=CONFIDENCE_THRESHOLD, verbose=False, half=torch.cuda.is_available())[0]
                    if len(sub_result.boxes) == 0:
                        continue  # ROI không chắc chắn là biển, bỏ qua

                # Đến đây chắc chắn ROI hợp lệ, kiểm tra sắc nét
                roi = enhance_roi(roi)
                sc = sharpness_score(roi)
                if sc > tracks[tid]["best_score"]:
                    tracks[tid]["best_score"] = sc
                    tracks[tid]["best_roi"] = roi

            idx += 1
            pbar.progress(min(idx / max(frames, 1), 1.0))

        cap.release()

        # Merge tracks trước OCR
        tracks = merge_tracks(tracks)

        # ================= OCR cho mỗi track =================
        unique_texts = []
        for tid, t in tracks.items():
            if t["best_roi"] is not None:
                try:
                    if gemini_model:
                        ocr_status.markdown("✅ **Đang dùng Gemini OCR**", unsafe_allow_html=True)
                        pil_roi = Image.fromarray(cv2.cvtColor(t["best_roi"], cv2.COLOR_BGR2RGB))
                        txt_gem = ocr_with_gemini(pil_roi)
                        txt_easy = ocr_with_easyocr(t["best_roi"])
                        # Chọn cái khớp regex tốt hơn
                        if re.match(r"^\d{2}-[A-Z]{1,2} \d{2,6}$", txt_gem):
                            txt = txt_gem
                        else:
                            txt = txt_easy
                        if not txt.strip():
                            ocr_status.markdown("⚠️ **Gemini vượt quota, chuyển EasyOCR**", unsafe_allow_html=True)
                            txt = txt_easy
                    else:
                        ocr_status.markdown("⚠️ **Chưa có API key, dùng EasyOCR**", unsafe_allow_html=True)
                        txt = ocr_with_easyocr(t["best_roi"])
                    t["text"] = txt.strip()
                except Exception as e:
                    ocr_status.markdown(f"⚠️ **Gemini lỗi ({e}), dùng EasyOCR**", unsafe_allow_html=True)
                    t["text"] = ocr_with_easyocr(t["best_roi"])

                if t["text"] and is_valid_vietnam_plate(t["text"]):
                    unique_texts.append(t["text"])
                else:
                    st.info(f"🚫 Bỏ qua chuỗi không hợp lệ: {t['text']}")

        # ================= PASS 2: GHI VIDEO + DEBUG =================
        cap = cv2.VideoCapture(temp_video_path)
        sort_tracker = Sort(max_age=MAX_AGE, min_hits=1, iou_threshold=IOU_TH)
        idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if idx % frame_interval == 0:
                result = model.predict(frame, conf=CONFIDENCE_THRESHOLD, half=torch.cuda.is_available())[0]
                dets = np.array([[*box.xyxy[0].cpu().numpy(), box.conf[0].cpu().numpy()] for box in result.boxes]) if len(result.boxes) > 0 else np.empty((0, 5))
            else:
                dets = np.empty((0, 5))

            trackers = sort_tracker.update(dets)

            for track in trackers:
                x1, y1, x2, y2, tid = map(int, track)
                # Match tid với tracks từ pass 1 bằng IoU
                best_tid, best_iou = None, 0.0
                for saved_tid, t in tracks.items():
                    if t["boxes_history"]:
                        boxes_np = np.array(t["boxes_history"])
                        last_box = np.mean(boxes_np, axis=0).astype(int)
                        ov = iou([x1, y1, x2, y2], last_box)
                        if ov > IOU_TH and ov > best_iou:
                            best_tid, best_iou = saved_tid, ov

                if best_tid is not None:
                    label = tracks[best_tid]["text"]
                    if tracks[best_tid]["boxes_history"]:
                        boxes_np = np.array(tracks[best_tid]["boxes_history"])
                        x1, y1, x2, y2 = np.mean(boxes_np, axis=0).astype(int)
                    # Debug hiển thị track + biển số
                    cv2.putText(frame, f"#{best_tid}", (x1, y2 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
                else:
                    label = ""

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                if label:
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            out.write(frame)
            idx += 1

        cap.release()
        out.release()

        # ================= HIỂN THỊ KẾT QUẢ =================
        if not box_found:
            st.warning("❌ Không có biển số được phát hiện trong video.")
        else:
            st.subheader("📥 Video gốc")
            col1, col2, col3 = st.columns([1, 2, 1])
            with col2:
                st.video(temp_video_path)

            st.subheader("🖼️ ROI tốt nhất mỗi track")
            for tid, t in tracks.items():
                if t["best_roi"] is not None and t["text"] and is_valid_vietnam_plate(t["text"]):
                    st.image(
                        t["best_roi"],
                        caption=f"🖼️ Track #{tid} (Biển số: {t['text']})",
                        width=250
                    )

            st.subheader("📥 Video đã nhận diện")
            with open(out_path, "rb") as f:
                st.download_button(
                    "📥 Tải video đã nhận diện",
                    f,
                    file_name="output_annotated.mp4"
                )

        for txt in unique_texts:
            plate_counter[txt] += 1

    # ===== KẾT QUẢ CUỐI =====
    st.subheader("📋 Biển số phát hiện được:")
    found_plate = False
    for plate, count in plate_counter.items():
        if plate.strip():
            st.write(f"- **{plate}** ({count} lần)")
            found_plate = True
    if not found_plate:
        st.warning("❌ Không có biển số được phát hiện.")

    # Export CSV
    if plate_counter:
        df = pd.DataFrame(list(plate_counter.items()), columns=['Biển số', 'Số lần'])
        csv = df.to_csv(index=False).encode('utf-8')
        st.download_button("📥 Tải CSV kết quả", csv, "plates.csv", "text/csv")

# ===== GHI CHÚ =====
    st.markdown(
        """
        <div style='position: fixed; bottom: 10px; right: 10px; font-size: 14px; color: gray;'>
           By Võ Công Nhật - 20222627.
        </div>
        """,
        unsafe_allow_html=True
    )

# Dọn file tạm (di chuyển xuống đây để sau download)
try:
    if 'temp_video' in locals() and os.path.exists(temp_video.name):
        os.unlink(temp_video.name)
    if 'out_path' in locals() and os.path.exists(out_path):
        os.unlink(out_path)
except:
    pass

Overwriting /content/project/app.py


In [34]:
!streamlit run /content/project/app.py --server.port 8501 &> /content/log.txt &


In [ ]:
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared
!./cloudflared tunnel --url http://localhost:8501


2025-08-26T02:53:09Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-08-26T02:53:09Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-08-26T02:53:12Z INF +--------------------------------------------------------------------------------------------+
2025-08-26T02:53:12Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-08-26T02:53:12Z INF |  https://titans-definitely-trainers-ot.trycloudflare.c